<a href="https://colab.research.google.com/github/gretiere545/corpus/blob/main/corpus_maker_init.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fabrique des Corpus ASAMLA

## Phase 1 : Initialisation d'un nouveau Corpus

In [1]:
!export PYTHONIOENCODING=utf8
!pip install gspread-formatting
import pandas as pd
import numpy as np
import base64
import uuid
import zlib
import binascii
import random
import os
import json
import secrets
pd.set_option("display.width",1000)
# Ce bout de code pour pouvoir downloader des fichiers sur le drive
from google.colab import drive
drive.mount('/content/drive')
# Ce bout de code pour pouvoir downloader des google sheets dans des dataframes
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_formatting import *
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
%cd /content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


Mounted at /content/drive
/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


In [2]:
def make_unique_id():
  return str(uuid.uuid4())[:8]

In [3]:
def format_feuille(wb, nom_onglet):
  # onglet
  ws = wb.worksheet(nom_onglet)
  # Format de la partie gauche
  fmt = cellFormat(
      backgroundColor=color(0.91, 0.96, 0.93),
      textFormat=textFormat(bold=False, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, 'A:B', fmt)

  # format de l'entete
  fmt = cellFormat(
      backgroundColor=color(0.7725,0.8431,0.7922),
      textFormat=textFormat(bold=True, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, '1', fmt)
  
  # on gèle l'entete et les colonnes de gauche
  set_frozen(ws, rows=1, cols=2)
  set_column_width(ws, 'A', 100)
  set_column_width(ws, 'B', 400)
  set_column_width(ws, 'C:', 600)

Référentiels Corpus de travail

In [29]:
# Corpus Français Médical Vaccination
corpus_fr_med_vac = "asamla-corpus_francais_med_vac-v1.2"
corpus_fr_med_vac_uri = "https://docs.google.com/spreadsheets/d/1-2SDHCFR5JOHCc4MiYSMo4V8yG7kfHbpIKV3q9ci6wE"
# Corpus Français Médical Médecine Générale
corpus_fr_med_gen = "ASAMLA - Corpus Médical Médecine Générale-Liste des expressions à traduire-v1.0"
corpus_fr_med_gen_uri = "https://docs.google.com/spreadsheets/d/1M3CFSpAGII25qCl0apYrRPX-DmpdxDFLLB_Ob-MX8DQ"

In [6]:
# Corpus Toutes Langues Médical Vaccination
corpus_trad_med_vac = "ASAMLA - Corpus Médical Vaccination-v2.0"
corpus_trad_med_vac_uri = "https://docs.google.com/spreadsheets/d/1CclzYfFCW4srA3Lq_np2LpSrxj84JpcbzytL449DH8E"
# Corpus Toutes Langues Médical Médecine Générale
corpus_trad_med_neu = "ASAMLA - Corpus Médical Médecine Générale-Traduction-v1.0"
corpus_trad_med_neu_uri = "https://docs.google.com/spreadsheets/d/1zB7hvjr2HufHcVgcJfEtrUUmR75wU5OCiL-_d4oDtTU"

Ouverture de la base centrale (repository) : base se trouvant sur gdrive atos.net

In [30]:
# Ouverture de la Sheet Centrale (corpus_central_base)
sheet_central = 'https://docs.google.com/spreadsheets/d/1L8YB1aXHUJwUE9AE6xyn_xMHalinGR335Q7lntwbu1U'
wb_central = gc.open_by_url(sheet_central)
# onglet Médical Vaccination
corpus_trad_med_vac_onglet = "med_vac_synthese"
# onglet Médical Médecine Générale
corpus_trad_med_gen_onglet = "med_gen_synthese"

Chargement du Corpus FR : à ce stade, il ne contient que des expressions en français

In [7]:
def get_cc_config(cfg):
  vk_lang_dict = []
  try:
    with open(cfg) as vk_dict:
        vk_lang_dict = json.load(vk_dict)
  except BaseException as e:
    print(e)
  return vk_lang_dict

def set_cc_config (vk_lang_dict, cfg):
  # sauvegarde du fichier de paramétrage
  with open(cfg, 'w') as fp:
      json.dump(vk_lang_dict, fp)  

In [8]:
lang_config = 'med_vac_synthese.json'
vk_lang_dict = get_cc_config(lang_config)

In [30]:
uri = corpus_fr_med_gen_uri

In [10]:
def narrow_no_break_space (s):
  s = " ".join(s.split())   # on vire tous les espaces en trop
  find = r'(\s*\?$)'        # on recherche le ? précédé ou pas d'espace \s*
  replace = u'\u202F'+ r'?'
  s = re.sub(find, replace, s)    # on remplace par un blanc insécable + le ?
  return s

2021 10 05 : 
Suppression du Peul, remplacé par Allemand

Fonction d'attribution d'id unique pour chaque expression à l'intérieur d'un corpus

In [35]:
#ouverture de l'onglet Corpus dans un DF
wb = gc.open_by_url(uri)
t_corpus = wb.get_worksheet(0)
data_t_corpus = t_corpus.get_all_values()
df_corpus = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0])

In [36]:
df_corpus

,uid,expression,traduction,date,commentaires,état
0,03bb6e86,"Bonjour, comment vous appelez-vous ?",,09/12/2021,,validé
1,38a24ddb,Quel est le motif de votre visite ?,,09/12/2021,,validé
2,ba312208,Quel âge avez-vous ?,,09/12/2021,,validé
3,192bc4c8,Vous venez de quel pays ?,,09/12/2021,,validé
4,955f030c,Buvez-vous de l’alcool ?,,09/12/2021,,validé
...,...,...,...,...,...,...
78,50460084,Saignement,,09/12/2021,,validé
79,af9faaa5,Purulent,,09/12/2021,,validé
80,1ffe71ca,Fatigue,,09/12/2021,,validé
81,6fa427fc,Angine,,09/12/2021,,validé


In [37]:
df_corpus['uid'] = df_corpus['uid'].astype("string")

In [41]:
df_corpus

,uid,français,traduction
0,03bb6e86,"Bonjour, comment vous appelez-vous ?",
1,38a24ddb,Quel est le motif de votre visite ?,
2,ba312208,Quel âge avez-vous ?,
3,192bc4c8,Vous venez de quel pays ?,
4,955f030c,Buvez-vous de l’alcool ?,
...,...,...,...
78,50460084,Saignement,
79,af9faaa5,Purulent,
80,1ffe71ca,Fatigue,
81,6fa427fc,Angine,


In [37]:
# SI L'état de l'expression est VALIDE on ajoute le hash qui sert de clé
df_corpus['uid'] = df_corpus[['uid','état']].apply(lambda x: x[0] if (x[0]!="" and x[1]=="validé") else make_unique_id() if (x[0]=="" and x[1]=="validé") else "", axis=1)
# exemple de bout de code pour accéder à un enregistrement par la clé
# df_corpus.loc[df_corpus['uid']=='g06g6WJ6ROb8-tWX2TlVtwvKY3MdWg']['coded_expression'].apply(lambda x:decode_expression(x))

In [39]:
# on élimine toutes les lignes sans expression
df_corpus["expression"].replace('', np.nan, inplace=True)
df_corpus = df_corpus[df_corpus['expression'].notna()]
# trim des expressions
df_corpus['français'] = df_corpus['expression'].apply(lambda x:x.strip())
df_corpus['français'] = df_corpus['français'].apply(lambda x:narrow_no_break_space(x))   

In [40]:
# on ne conserve que les colonnes utiles
df_corpus = df_corpus[['uid','français']]
# on ajoute la colonne pour la traduction
df_corpus['traduction'] = ''

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Sauvegarde du corpus FR avec l'uid

In [42]:
# sauvegarde du dataframe dans la table wb_central de la feuille gsheet
set_with_dataframe(t_corpus, df_corpus)


In [ ]:
df_corpus = df_corpus.loc[df_corpus['uid']!=""]

#### Déploiement du corpus français dans un corpus de traduction : 1 langue = 1 onglet

In [43]:
uri = corpus_trad_med_neu_uri
#ouverture de l'onglet Corpus dans un DF
wb = gc.open_by_url(uri)
#pour chaque corpus, on créé un onglet si besoin
for index, item in enumerate(vk_lang_dict):
  #df_corpus_all.append (df_corpus)
  nom_onglet = item['language']
  print ("Processing " + nom_onglet)  
  # si l'onglet existe déjà, on le supprime ?
  try:
    ws = wb.worksheet(nom_onglet)
    wb.del_worksheet(ws)  
  except:
    print ("Onglet inexistant !")
    print ("Non existing : " + nom_onglet)
  wb.add_worksheet(nom_onglet, 1, 1)
  export_sheet = wb.worksheet(nom_onglet)
  set_with_dataframe(export_sheet, df_corpus)
  format_feuille(wb, nom_onglet)    

Processing Arabe
Processing Anglais
Processing Turc
Processing Russe
Processing Ukrainien
Processing Roumain
Processing Hongrois
Processing Tigrinya
Processing Albanais
Processing Géorgien
Processing Arménien
Processing Dari
Processing Pashto
Processing Fârsi
Processing Azéri
Processing Espagnol
Processing Amharique
Processing Allemand
Processing Polonais
Onglet inexistant !
Non existing : Polonais
Processing Somali
Onglet inexistant !
Non existing : Somali


In [ ]:
uri = corpus_trad_med_neu_uri
#ouverture de l'onglet Corpus dans un DF
wb = gc.open_by_url(uri)
#pour chaque corpus, on créé un onglet si besoin
for index, item in enumerate(vk_lang_dict):
  #df_corpus_all.append (df_corpus)
  nom_onglet = item['language']
  print ("Processing " + nom_onglet)
  # si l'onglet existe déjà, on le supprime ?
  try:
    ws = wb.worksheet(nom_onglet)
    data_onglet = ws.get_all_values()
    df_onglet = pd.DataFrame(data_onglet[1:], columns=data_onglet[0])
    # fusion des données présentes et nouvelles
    frames = [df_onglet, df_corpus]
    result = pd.concat([df_onglet, df_corpus]).drop_duplicates('uid').reset_index(drop=True)
    set_with_dataframe(ws, result)
    format_feuille(wb, nom_onglet)     
  except:
    print ("Onglet inexistant !")
    print ("Non existing : " + nom_onglet)
    wb.add_worksheet(nom_onglet, 1, 1)
    export_sheet = wb.worksheet(nom_onglet)
    set_with_dataframe(export_sheet, df_corpus)
    format_feuille(wb, nom_onglet)   

In [15]:
nom_onglet = 'Turc'
ws = wb.worksheet(nom_onglet)
data_onglet = ws.get_all_values()
df_onglet = pd.DataFrame(data_onglet[1:], columns=data_onglet[0])

In [18]:
pd.merge(left=df_corpus, right=df_onglet, 
                      how='outer',
                      left_index=True,
                      right_index=True,
                      on=('uid')
                     ).sort_index(axis=1)

,commentaires_x,commentaires_y,date_x,date_y,français_x,français_y,traduction_x,traduction_y,uid,état_x,état_y
0,,,09/12/2021,,"Bonjour, comment vous appelez-vous ?","Bonjour, comment vous appelez-vous ?",,Merhaba isminiz nedir ?,03bb6e86,validé,
1,,,09/12/2021,,Quel est le motif de votre visite ?,Quel est le motif de votre visite ?,,niçin burdasınız?,38a24ddb,validé,
2,,,09/12/2021,,Quel âge avez-vous ?,Quel âge avez-vous ?,,kaç yaşındasınız,ba312208,validé,
3,,,09/12/2021,,Vous venez de quel pays ?,Vous venez de quel pays ?,,hangi ûlkeden geliyorsunuz?,192bc4c8,validé,
4,,,09/12/2021,,Buvez-vous de l’alcool ?,Buvez-vous de l’alcool ?,,alkol içiyormusunuz?,955f030c,validé,
...,...,...,...,...,...,...,...,...,...,...,...
79,,,09/12/2021,,Purulent,Purulent,,iltihaplı,af9faaa5,validé,
80,,,09/12/2021,,Fatigue,Fatigue,,yorgunluk,1ffe71ca,validé,
81,,,09/12/2021,,Angine,angine,,,6fa427fc,validé,
82,,,09/12/2021,,Œdème,oedème,,,72f0e8db,validé,


In [19]:
df_onglet

,uid,français,traduction,date,commentaires,état
0,03bb6e86,"Bonjour, comment vous appelez-vous ?",Merhaba isminiz nedir ?,,,
1,38a24ddb,Quel est le motif de votre visite ?,niçin burdasınız?,,,
2,ba312208,Quel âge avez-vous ?,kaç yaşındasınız,,,
3,192bc4c8,Vous venez de quel pays ?,hangi ûlkeden geliyorsunuz?,,,
4,955f030c,Buvez-vous de l’alcool ?,alkol içiyormusunuz?,,,
...,...,...,...,...,...,...
79,af9faaa5,Purulent,iltihaplı,,,
80,1ffe71ca,Fatigue,yorgunluk,,,
81,6fa427fc,angine,,,,
82,72f0e8db,oedème,,,,


In [22]:
new_df = df_onglet.merge(df_corpus, on='uid', how='left')
new_df.drop(['français_x', 'date_x','commentaires_x','traduction_y','état_x'], axis=1, inplace=True)
new_df.rename(columns={'traduction_x': 'traduction', 'français_y': 'français','date_y': 'date', 'commentaires_y': 'commentaires', 'état_y': 'état'})

,uid,traduction,français,date,commentaires,état
0,03bb6e86,Merhaba isminiz nedir ?,"Bonjour, comment vous appelez-vous ?",09/12/2021,,validé
1,38a24ddb,niçin burdasınız?,Quel est le motif de votre visite ?,09/12/2021,,validé
2,ba312208,kaç yaşındasınız,Quel âge avez-vous ?,09/12/2021,,validé
3,192bc4c8,hangi ûlkeden geliyorsunuz?,Vous venez de quel pays ?,09/12/2021,,validé
4,955f030c,alkol içiyormusunuz?,Buvez-vous de l’alcool ?,09/12/2021,,validé
...,...,...,...,...,...,...
79,af9faaa5,iltihaplı,Purulent,09/12/2021,,validé
80,1ffe71ca,yorgunluk,Fatigue,09/12/2021,,validé
81,6fa427fc,,Angine,09/12/2021,,validé
82,72f0e8db,,Œdème,09/12/2021,,validé


In [16]:
df_onglet['français']=

,uid,français,traduction,date,commentaires,état
0,03bb6e86,"Bonjour, comment vous appelez-vous ?",Merhaba isminiz nedir ?,,,
1,38a24ddb,Quel est le motif de votre visite ?,niçin burdasınız?,,,
2,ba312208,Quel âge avez-vous ?,kaç yaşındasınız,,,
3,192bc4c8,Vous venez de quel pays ?,hangi ûlkeden geliyorsunuz?,,,
4,955f030c,Buvez-vous de l’alcool ?,alkol içiyormusunuz?,,,
...,...,...,...,...,...,...
79,af9faaa5,Purulent,iltihaplı,,,
80,1ffe71ca,Fatigue,yorgunluk,,,
81,6fa427fc,angine,,,,
82,72f0e8db,oedème,,,,
